In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Graph Convolutional Networks in PyTorch
https://github.com/tkipf/pygcn
    

In [1]:
# utils.py

import numpy as np
import scipy.sparse as sp
import torch
import math

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [2]:
# 


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [3]:
# models.py

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)


In [5]:
# train.py

from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

# Training settings
no_cuda = False
fastmode = False
seed = 42
epochs = 200
lr = 0.01
weight_decay = 5e-4
hidden = 16
dropout = 0.5

cuda = not no_cuda and torch.cuda.is_available()

np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Loading cora dataset...
Epoch: 0001 loss_train: 1.9548 acc_train: 0.1500 loss_val: 1.9546 acc_val: 0.1567 time: 0.2533s
Epoch: 0002 loss_train: 1.9429 acc_train: 0.1929 loss_val: 1.9425 acc_val: 0.1567 time: 0.0338s
Epoch: 0003 loss_train: 1.9334 acc_train: 0.2000 loss_val: 1.9315 acc_val: 0.1567 time: 0.0217s
Epoch: 0004 loss_train: 1.9142 acc_train: 0.2000 loss_val: 1.9204 acc_val: 0.1567 time: 0.0226s
Epoch: 0005 loss_train: 1.8960 acc_train: 0.2000 loss_val: 1.9098 acc_val: 0.1567 time: 0.0211s
Epoch: 0006 loss_train: 1.8925 acc_train: 0.2000 loss_val: 1.8995 acc_val: 0.1567 time: 0.0217s
Epoch: 0007 loss_train: 1.8818 acc_train: 0.2000 loss_val: 1.8897 acc_val: 0.1567 time: 0.0187s
Epoch: 0008 loss_train: 1.8837 acc_train: 0.2000 loss_val: 1.8804 acc_val: 0.1567 time: 0.0199s
Epoch: 0009 loss_train: 1.8660 acc_train: 0.2000 loss_val: 1.8714 acc_val: 0.1567 time: 0.0191s
Epoch: 0010 loss_train: 1.8547 acc_train: 0.2000 loss_val: 1.8624 acc_val: 0.1567 time: 0.0190s
Epoch: 0011 loss

Epoch: 0093 loss_train: 0.8544 acc_train: 0.8000 loss_val: 1.0311 acc_val: 0.7700 time: 0.0187s
Epoch: 0094 loss_train: 0.8037 acc_train: 0.8286 loss_val: 1.0233 acc_val: 0.7733 time: 0.0211s
Epoch: 0095 loss_train: 0.8444 acc_train: 0.7929 loss_val: 1.0153 acc_val: 0.7700 time: 0.0273s
Epoch: 0096 loss_train: 0.8794 acc_train: 0.7714 loss_val: 1.0082 acc_val: 0.7733 time: 0.0266s
Epoch: 0097 loss_train: 0.8033 acc_train: 0.8500 loss_val: 1.0014 acc_val: 0.7733 time: 0.0253s
Epoch: 0098 loss_train: 0.8183 acc_train: 0.7929 loss_val: 0.9938 acc_val: 0.7800 time: 0.0214s
Epoch: 0099 loss_train: 0.7677 acc_train: 0.8500 loss_val: 0.9858 acc_val: 0.7800 time: 0.0234s
Epoch: 0100 loss_train: 0.7993 acc_train: 0.8214 loss_val: 0.9789 acc_val: 0.7800 time: 0.0261s
Epoch: 0101 loss_train: 0.7795 acc_train: 0.8286 loss_val: 0.9713 acc_val: 0.7800 time: 0.0283s
Epoch: 0102 loss_train: 0.7801 acc_train: 0.8286 loss_val: 0.9639 acc_val: 0.7833 time: 0.0260s
Epoch: 0103 loss_train: 0.7461 acc_train

Epoch: 0182 loss_train: 0.4314 acc_train: 0.9214 loss_val: 0.7082 acc_val: 0.8067 time: 0.0182s
Epoch: 0183 loss_train: 0.4213 acc_train: 0.9571 loss_val: 0.7062 acc_val: 0.8100 time: 0.0189s
Epoch: 0184 loss_train: 0.3942 acc_train: 0.9357 loss_val: 0.7046 acc_val: 0.8100 time: 0.0186s
Epoch: 0185 loss_train: 0.3907 acc_train: 0.9429 loss_val: 0.7034 acc_val: 0.8100 time: 0.0193s
Epoch: 0186 loss_train: 0.4236 acc_train: 0.9286 loss_val: 0.7035 acc_val: 0.8100 time: 0.0173s
Epoch: 0187 loss_train: 0.4514 acc_train: 0.9143 loss_val: 0.7049 acc_val: 0.8033 time: 0.0160s
Epoch: 0188 loss_train: 0.4297 acc_train: 0.9143 loss_val: 0.7047 acc_val: 0.8100 time: 0.0204s
Epoch: 0189 loss_train: 0.3762 acc_train: 0.9357 loss_val: 0.7040 acc_val: 0.8100 time: 0.0164s
Epoch: 0190 loss_train: 0.3895 acc_train: 0.9286 loss_val: 0.7051 acc_val: 0.8067 time: 0.0181s
Epoch: 0191 loss_train: 0.4237 acc_train: 0.9571 loss_val: 0.7047 acc_val: 0.8067 time: 0.0178s
Epoch: 0192 loss_train: 0.4496 acc_train